# IMPORT PACKAGE



In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime
from google.colab import files

In [ ]:
# logging instead of print
import logging
logging.basicConfig(filename='pipeline.log',format='%(levelname)s: %(message)s', level=logging.INFO, force=True)
logger = logging.getLogger()

# LOAD AND VALIDATE DATA




1.   Load the necessary data and correct the formate.
2.   Check for columns : 'call_date', 'agent_id',
     'org_id' present in the dataset.
3.   Flag missing or duplicate entries.



In [ ]:
def load_data():
    try:
        # Load the data
        call_logs = pd.read_csv("/content/call_logs.csv")
        agent_roster = pd.read_csv("/content/agent_roster.csv")
        disposition_summary = pd.read_csv("/content/disposition_summary.csv")

        # Convert the datetime columns
        call_logs["created_ts"] = pd.to_datetime(call_logs["created_ts"].str.replace("T", " "),format="%Y-%m-%d %H:%M:%S")
        call_logs["call_date"] = pd.to_datetime(call_logs["call_date"], format="%Y-%m-%d")

        disposition_summary["call_date"] = pd.to_datetime(disposition_summary["call_date"], format="%Y-%m-%d")
        disposition_summary["login_time"] = pd.to_datetime(disposition_summary["login_time"], format="%H:%M")

        logging.info("Data files loaded and parsed successfully.")
        return call_logs, agent_roster, disposition_summary

    except Exception as e:
        logging.error(f"Error loading files: {e}")
        raise


In [ ]:
call_logs, agent_roster, disposition_summary = load_data()

In [ ]:
# Check for presence of necessary columns.

def validate_required_columns(call_logs, disposition_summary, agent_roster):
    required_columns = ['call_date', 'agent_id', 'org_id']
    datasets = {
        'Call Logs': call_logs,
        'Disposition Summary': disposition_summary,
        'Agent Roster': agent_roster
    }

    for name, df in datasets.items():
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            logging.warning(f"{name} is missing columns: {missing_columns}")
        else:
            logging.info(f"All required columns present in {name}.")

In [ ]:
validate_required_columns(call_logs,disposition_summary,agent_roster)

In [ ]:
# Check for missing rows or duplicate columns.

def flag_missing_and_duplicates(call_logs, agent_roster, disposition_summary):
    datasets = {
        "Call Logs": call_logs,
        "Agent Roster": agent_roster,
        "Disposition Summary": disposition_summary
    }

    for name, df in datasets.items():
        # Check for missing values
        missing_rows = df[df.isnull().any(axis=1)]
        if not missing_rows.empty:
            logging.warning("\n Missing values in %s:\n%s\n", name, missing_rows)
        else:
            logging.info("\n No missing values in %s:\n",name)

    for name, df in datasets.items():
        # Check for duplicate rows
        duplicate_rows = df[df.duplicated()]
        if not duplicate_rows.empty:
            logging.warning("\n Duplicate rows in %s:\n%s\n", name, duplicate_rows)
        else:
            logging.info("\n No duplicate rows in %s:\n",name)


In [ ]:
flag_missing_and_duplicates(call_logs, agent_roster, disposition_summary)

In [ ]:
print(call_logs.info(),"\n")
print(agent_roster.info(),"\n")
print(disposition_summary.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   call_id         500 non-null    object        
 1   agent_id        500 non-null    object        
 2   org_id          500 non-null    object        
 3   installment_id  500 non-null    object        
 4   status          500 non-null    object        
 5   duration        500 non-null    float64       
 6   created_ts      500 non-null    datetime64[ns]
 7   call_date       500 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 31.4+ KB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   agent_id               20 non-null     object
 1   users_first_name       20 non-null   

# MERGE DATASET (JOIN)


In [ ]:
# Merge Data
def merge_data(call_logs, agent_roster, disposition_summary):

    # merge1 -> 'call_logs' & 'agent_roster' based on 'agent_id' & 'org_id' using LEFT join

    df = call_logs.merge(agent_roster,
                         on=["agent_id", "org_id"],
                         how="left")

    # merge2 -> 'merge1' & 'disposition_summary' based on 'agent_id', 'org_id' & 'call_date' using LEFT join

    df = df.merge(disposition_summary,
                  on=["agent_id", "org_id", "call_date"],
                  how="left")

    # check for not matched rows
    unmatched = df[df['users_first_name'].isnull()]
    if not unmatched.empty:
        logging.warning(f"{len(unmatched)} agents could not be matched with the roster.")

    logging.info("Data merged successfully.")
    return df


In [ ]:
final_data = merge_data(call_logs, agent_roster, disposition_summary)
final_data.head(10)

,call_id,agent_id,org_id,installment_id,status,duration,created_ts,call_date,users_first_name,users_last_name,users_office_location,login_time
0,C5333,A020,O2,L1826,completed,5.68,2025-04-28 15:40:00,2025-04-28,AgentFirst20,AgentLast20,Delhi,1900-01-01 09:50:00
1,C3045,A018,O1,L1996,no_answer,14.27,2025-04-28 02:41:00,2025-04-28,AgentFirst18,AgentLast18,Bangalore,NaT
2,C5803,A018,O1,L1849,failed,11.01,2025-04-28 19:42:00,2025-04-28,AgentFirst18,AgentLast18,Bangalore,NaT
3,C2139,A007,O1,L1046,connected,9.02,2025-04-28 09:52:00,2025-04-28,AgentFirst7,AgentLast7,Bangalore,1900-01-01 08:36:00
4,C4814,A003,O1,L1887,completed,2.42,2025-04-28 12:58:00,2025-04-28,AgentFirst3,AgentLast3,Mumbai,1900-01-01 10:24:00
5,C8428,A009,O3,L1650,failed,2.42,2025-04-28 05:33:00,2025-04-28,AgentFirst9,AgentLast9,Bangalore,1900-01-01 11:46:00
6,C6820,A012,O3,L1214,failed,0.97,2025-04-28 23:57:00,2025-04-28,AgentFirst12,AgentLast12,Chennai,NaT
7,C3803,A003,O1,L1546,connected,13.01,2025-04-28 05:34:00,2025-04-28,AgentFirst3,AgentLast3,Mumbai,1900-01-01 10:24:00
8,C7216,A015,O2,L1276,connected,9.06,2025-04-28 23:22:00,2025-04-28,AgentFirst15,AgentLast15,Chennai,1900-01-01 08:05:00
9,C1916,A011,O1,L1234,completed,10.65,2025-04-28 10:46:00,2025-04-28,AgentFirst11,AgentLast11,Chennai,1900-01-01 11:08:00


In [ ]:
final_data.shape

(500, 12)

# FEATURE ENGINEERING

For each agent:

1.   Total Calls made.
2.   Unique Loans contacted.
3.   Connect Rate.
4.   Average call duration.
5.   Presence (1 if login_time exists, else 0).



In [ ]:
def compute_metrics(df):
    df['is_completed'] = df['status'].str.lower() == 'completed'
    df['duration_min'] = df['duration'] / 60
    df['presence'] = df['login_time'].notnull().astype(int)

    summary = df.groupby(
        ['agent_id', 'call_date', 'users_first_name', 'users_last_name']
    ).agg(
        total_calls=('call_id', 'count'),
        unique_loans=('installment_id', pd.Series.nunique),
        completed_calls=('is_completed', 'sum'),
        avg_duration=('duration_min', 'mean'),
        login=('presence', 'max')
    ).reset_index()

    summary['connect_rate'] = (summary['completed_calls'] / summary['total_calls']).round(2)
    summary['avg_duration'] = summary['avg_duration'].round(2)

    logging.info("Metrics computed successfully.")
    return summary


In [ ]:
computed_metrics = compute_metrics(final_data)
computed_metrics

,agent_id,call_date,users_first_name,users_last_name,total_calls,unique_loans,completed_calls,avg_duration,login,connect_rate
0,A001,2025-04-28,AgentFirst1,AgentLast1,20,20,2,0.11,1,0.10
1,A002,2025-04-28,AgentFirst2,AgentLast2,23,23,3,0.13,1,0.13
2,A003,2025-04-28,AgentFirst3,AgentLast3,21,21,8,0.12,1,0.38
3,A004,2025-04-28,AgentFirst4,AgentLast4,27,27,4,0.13,1,0.15
4,A005,2025-04-28,AgentFirst5,AgentLast5,29,28,4,0.12,0,0.14
5,A006,2025-04-28,AgentFirst6,AgentLast6,16,16,2,0.15,1,0.12
6,A007,2025-04-28,AgentFirst7,AgentLast7,24,23,5,0.12,1,0.21
7,A008,2025-04-28,AgentFirst8,AgentLast8,29,29,8,0.11,1,0.28
8,A009,2025-04-28,AgentFirst9,AgentLast9,26,25,7,0.12,1,0.27
9,A010,2025-04-28,AgentFirst10,AgentLast10,38,38,7,0.13,1,0.18


# SLACK-STYLE SUMMARY

In [ ]:
def generate_slack_summary(computed_metrics, report_date):

    date_df = computed_metrics[computed_metrics["call_date"] == report_date]

    if date_df.empty:
        logging.warning(f"No data available for {report_date}")
        return

    # Handle missing agent name if not merged already
    if "agent_name" not in date_df.columns:
        date_df["agent_name"] = date_df["agent_id"]


    # Top performer
    top_agent = date_df.sort_values("connect_rate", ascending=False).iloc[0]
    top_agent_name = f"{top_agent['users_first_name']} {top_agent['users_last_name']}"
    agent_connect_rate = round(top_agent["connect_rate"] * 100)
    active = date_df['login'].sum()
    avg_dur = date_df['avg_duration'].mean().round(2)


    # summary
    summary = f"""
    Agent Summary for : {report_date}
    Top Performer: {top_agent_name} ({agent_connect_rate}% connect rate)
    Total Active Agents: {active}
    Average Duration: {avg_dur} min """

    return summary


In [ ]:
logging.info(generate_slack_summary(computed_metrics, report_date="2025-04-28"))


In [ ]:
# performance of all in decending order

computed_metrics.sort_values(by="connect_rate", ascending=False)

,agent_id,call_date,users_first_name,users_last_name,total_calls,unique_loans,completed_calls,avg_duration,login,connect_rate
2,A003,2025-04-28,AgentFirst3,AgentLast3,21,21,8,0.12,1,0.38
19,A020,2025-04-28,AgentFirst20,AgentLast20,21,21,7,0.11,1,0.33
13,A014,2025-04-28,AgentFirst14,AgentLast14,30,30,10,0.13,1,0.33
7,A008,2025-04-28,AgentFirst8,AgentLast8,29,29,8,0.11,1,0.28
15,A016,2025-04-28,AgentFirst16,AgentLast16,22,22,6,0.15,1,0.27
8,A009,2025-04-28,AgentFirst9,AgentLast9,26,25,7,0.12,1,0.27
11,A012,2025-04-28,AgentFirst12,AgentLast12,20,20,5,0.12,0,0.25
10,A011,2025-04-28,AgentFirst11,AgentLast11,22,22,5,0.15,1,0.23
18,A019,2025-04-28,AgentFirst19,AgentLast19,27,27,6,0.12,1,0.22
6,A007,2025-04-28,AgentFirst7,AgentLast7,24,23,5,0.12,1,0.21


# SAVE FILE

In [ ]:
output_path = "agent_performance_summary.csv"
computed_metrics.to_csv(output_path, index=False)
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#contents of the log file
with open('pipeline.log', 'r') as file:
    print(file.read())

#download log file
files.download('pipeline.log')

INFO: Data files loaded and parsed successfully.
INFO: All required columns present in Call Logs.
INFO: All required columns present in Disposition Summary.
INFO: 
 No missing values in Call Logs:

INFO: 
 No missing values in Agent Roster:

 Missing values in Disposition Summary:
   agent_id org_id  call_date login_time
4      A005     O3 2025-04-28        NaT
11     A012     O3 2025-04-28        NaT
17     A018     O1 2025-04-28        NaT

INFO: 
 No duplicate rows in Call Logs:

INFO: 
 No duplicate rows in Agent Roster:

INFO: 
 No duplicate rows in Disposition Summary:

INFO: Data merged successfully.
INFO: Metrics computed successfully.
INFO: 
    Agent Summary for : 2025-04-28
    Top Performer: AgentFirst3 AgentLast3 (38% connect rate)
    Total Active Agents: 17
    Average Duration: 0.12 min 



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>